# Quantum Algorithms

In [1]:
from lib.QuantumCircuit import*
from numpy import array, kron, pi, exp, sqrt, sin, cos, eye, zeros
from functools import reduce

In [2]:
X = array([[0, 1], [1, 0]])
Y = array([[0, -1j], [1j, 0]])
Z = array([[1, 0], [0, -1]])
H = (X + Z)/sqrt(2)
T = array([[1, 0], [0, exp(1j * pi/4)]])
S = array([[1, 0], [0, 1j]])

## Deutsch Algorithm

In [3]:
def Deutsch():
        
        o1 = generateQuantumGate(n = 4, free_bits_dict={0: 'X', 2: 'X'})
        o2 = generateQuantumGate(n = 4, controlled_bits_dict={0: 1}, target_bits_dict={3: 'X'})
        o3 = generateQuantumGate(n = 4, controlled_bits_dict={1: 1}, target_bits_dict={3: 'X'})
        o4 = generateQuantumGate(n = 4, controlled_bits_dict={2: 1}, target_bits_dict={3: 'X'})
        o5 = generateQuantumGate(n = 4, free_bits_dict={0: 'X', 2: 'X'})
        oracle = reduce(lambda g1, g2: g1 @ g2, reversed([o1, o2, o3, o4, o5]))
        oracle = oracle.simplify()


        def oracle_output(state_tup):

            state0 = state_tup + (0,)
            state1 = state_tup + (1,) 

            out0 = (oracle @ SingleQuantumState(label_tup=state0)).evaluate()
            out1 = (oracle @ SingleQuantumState(label_tup=state1)).evaluate()

            if out0.label_tup[-1] == 0 and out1.label_tup[-1] == 1:
                return 0
            elif out0.label_tup[-1] == 1 and out1.label_tup[-1] == 0:
                return 1

        func_map = { (a, b, c): oracle_output((a, b, c)) for a, b, c in product(range(2), repeat=3)}
        print(func_map)

        state1 = SingleQuantumState(label_tup=(0, 0, 0, 0), coefficient=1/sqrt(2)) - SingleQuantumState(label_tup=(0, 0, 0, 1), coefficient=1/sqrt(2))
        state1 = (oracle @ state1).evaluate()
        print(state1)

        state2 = SingleQuantumState(label_tup=(0, 0, 1, 0), coefficient=1/sqrt(2)) - SingleQuantumState(label_tup=(0, 0, 1, 1), coefficient=1/sqrt(2))
        state2 = (oracle @ state2).evaluate()
        print(state2)

        def solution():

            g1 = generateQuantumGate(n = 4, free_bits_dict={0: 'H', 1: 'H', 2: 'H', 3: 'X'}, additional_operator_dict={'H': H})
            g2 = generateQuantumGate(n = 4, free_bits_dict={3: 'H'}, additional_operator_dict={'H': H})
            g3 = oracle
            g4 = generateQuantumGate(n = 4, free_bits_dict={0: 'H', 1: 'H', 2: 'H'}, additional_operator_dict={'H': H})
            circuit = constructQuantumCircuit(g1, g2, g3, g4)#reduce(lambda g1, g2: g1 @ g2, reversed([g1, g2, g3, g4]))
            circuit = circuit.simplify()

            state = SingleQuantumState(label_tup=(0, 0, 0, 0))

            state = (circuit @ state).evaluate()
            
            state = [s for s in state.rep if abs(s.coefficient) > 1e-10]

            return sum(state)
        
        print('-------- solution ----------')
        print(solution())

In [15]:
Deutsch()

{(0, 0, 0): 0, (0, 0, 1): 1, (0, 1, 0): 1, (0, 1, 1): 0, (1, 0, 0): 1, (1, 0, 1): 0, (1, 1, 0): 0, (1, 1, 1): 1}
0.7071067811865475 * (('I', 'I', 'I', 'I'),) @ (0, 0, 0, 0)
-0.7071067811865475 * (('I', 'I', 'I', 'I'),) @ (0, 0, 0, 1)

-0.7071067811865475 * (('I', 'I', 'I', 'I'),) @ (0, 0, 1, 0)
0.7071067811865475 * (('I', 'I', 'I', 'I'),) @ (0, 0, 1, 1)

-------- solution ----------
(-0.7071067812807148+0j) * (('I', 'I', 'I', 'I'),) @ (1, 1, 1, 1)
(0.7071067812807148+0j) * (('I', 'I', 'I', 'I'),) @ (1, 1, 1, 0)



## Bernstein Vazirani Algorithm

In [13]:
def Bernstein_Vazirani():
    g1 = generateQuantumGate(n = 4, free_bits_dict={0: 'H', 1: 'H', 2: 'H', 3: 'H'}, additional_operator_dict={'H': H})
    g2 = generateQuantumGate(n = 4, free_bits_dict={3: 'Z'})
    g3 = generateQuantumGate(n = 4, controlled_bits_dict={0: 1}, target_bits_dict={3: 'X'})
    g4 = generateQuantumGate(n = 4, controlled_bits_dict={1: 1}, target_bits_dict={3: 'X'})
    g5 = generateQuantumGate(n = 4, free_bits_dict={0: 'H', 1: 'H', 2: 'H'}, additional_operator_dict={'H': H})
    circuit = constructQuantumCircuit(g1, g2, g3, g4, g5)
    circuit = circuit.simplify()

    print(circuit)

    state = SingleQuantumState(label_tup=(0, 0, 0, 0))
    state = (circuit @ state).evaluate()
    state = [s for s in state.rep if abs(s.coefficient) > 1e-10]
    state = sum(state)
    print(state)  


In [14]:
Bernstein_Vazirani()

(('X', 'X', 'I', 'X'),): (-0.3535533906403574+0j)
(('X', 'X', 'I', 'I'),): (0.3535533906403574+0j)
(('I', 'I', 'I', 'I'),): 0.3535533906403574
(('I', 'I', 'I', 'Z'),): (-0.3535533906403574+0j)
(('I', 'I', 'I', 'Y'),): 0.3535533906403574j
(('I', 'I', 'I', 'X'),): (0.3535533906403574+0j)
(('X', 'X', 'I', 'Y'),): 0.3535533906403574j
(('X', 'X', 'I', 'Z'),): (0.3535533906403574+0j)

(-0.7071067812807148+0j) * (('I', 'I', 'I', 'I'),) @ (1, 1, 0, 1)
(0.7071067812807148+0j) * (('I', 'I', 'I', 'I'),) @ (1, 1, 0, 0)

